In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes transformers==4.31.0 trl==0.4.7


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install xformers


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import torch
import re
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
from huggingface_hub import login
login()

In [5]:
model_name = "meta-llama/Meta-Llama-3-8B"
dataset_name = "timdettmers/openassistant-guanaco"
new_model = "Llama-2-7b-chat-finetune"

lora_r = 16
lora_alpha = 8
lora_dropout = 0.1

use_8bit = True                                     
bnb_8bit_compute_dtype = "bf16"
bnb_8bit_quant_type = "nf8"

output_dir = "./results"
num_train_epochs = 1

fp16 = False
bf16 = True

per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 4
gradient_checkpointing = True
max_grad_norm = 0.25                              #check this out

learning_rate = 5*1e-5
weight_decay = 0.00001

optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"

max_steps = -1
warmup_ratio = 0.05                              #check this too
group_by_length = True
save_steps = 0
logging_steps = 5

max_seq_length = None
packing = False

device_map = {"": 0}

In [6]:
ds = load_dataset(dataset_name)

Repo card metadata block was not found. Setting CardData to empty.


In [7]:
def make_data(raw_data):
    
    sys_prompt = "<|start_header_id|>system<|end_header_id|>You are a helpful AI chatbot to humans<|eot_id|>"

    text = raw_data['text']
    formatted_chat = ""

    first_pre = "### Human:"
    second_pre = "### Assistant:"
 
    flist = [m.start() for m in re.finditer(first_pre, text)] 
    flist.append(len(text))
    slist = [m.start() for m in re.finditer(second_pre, text)] 


    for i in range(min(len(flist), len(slist))):
        
        human = text[flist[i]:slist[i]]
        chatbot = text[slist[i]:flist[i + 1]]
        human = human[human.index(':') + 1: ]
        chatbot = chatbot[chatbot.index(':') + 1: ]

        human = f"<|start_header_id|>Human<|end_header_id|>{human}<|eot_id|>"
        chatbot = f"<|start_header_id|>Chatbot<|end_header_id|>{chatbot}<|eot_id|>"

        formatted_chat += (human + chatbot)
    
    final_prompt = f"<|begin_of_text|> {sys_prompt} {formatted_chat} <|end_of_text|>"
    
    return {"text" : ''.join(final_prompt)}

In [8]:
data = ds['train'].map(make_data)
data = data.shuffle(seed=69)

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=use_8bit,
    bnb_8bit_quant_type=bnb_8bit_quant_type,
    bnb_8bit_compute_dtype=torch.bfloat16,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    quantization_config=bnb_config
)

In [11]:
model.config.use_cache = False
model.config.pretraining_tp = 1

In [12]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [14]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    bf16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [15]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [16]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} t

Step,Training Loss
5,7.799500
10,7.753100
15,7.396500
20,7.668900
25,7.326700
30,6.861200
35,6.467100
40,5.995200
45,5.180400
50,3.683300


In [ ]:
import pickle 
file_path = './base.pkl'

with open(file_path, 'wb') as files:
    pickle.dump(model.state_dict(), files)